<a href="https://colab.research.google.com/github/aramasethu/CS_573_project/blob/main/Merge_DM_Aishwarya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting google drive

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Mobility data

In [3]:
mobility = pd.read_csv (r'/content/drive/MyDrive/CS_573_Group_project/Data/Global_Mobility_Report.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
mobility_country1 = mobility[mobility['sub_region_1'].isna()]
mobility_country = mobility_country1[mobility_country1['metro_area'].isna()]

In [5]:
mobility_country = mobility_country.drop(columns=['country_region_code','sub_region_1', 'sub_region_2','metro_area', 'iso_3166_2_code','census_fips_code','place_id'])

In [6]:
mobility_country['date'] = mobility_country['date'].astype('datetime64[ns]')

In [7]:
mobility_country = mobility_country.rename(columns={ 'country_region': 'country'})

## Vaccination data loading

In [8]:
vec_cases = pd.read_csv(r'/content/drive/MyDrive/CS_573_Group_project/Data/owid-covid-data.csv')

In [9]:
vec_cases_scraped = vec_cases[['location', 'date', 'total_cases', 'total_deaths', 'population',
                              'new_vaccinations_smoothed', 'new_people_vaccinated_smoothed']]

In [10]:
vec_cases_new = vec_cases_scraped.copy()

In [11]:
vec_cases_new['total_cases'] = vec_cases_new['total_cases'].replace(np.nan, 0)
vec_cases_new['total_deaths'] = vec_cases_new['total_deaths'].replace(np.nan, 0)
vec_cases_new['new_vaccinations_smoothed'] = vec_cases_new['new_vaccinations_smoothed'].replace(np.nan, 0)
vec_cases_new['new_people_vaccinated_smoothed'] = vec_cases_new['new_people_vaccinated_smoothed'].replace(np.nan, 0)

In [12]:
vec_cases_new['date'] = pd.to_datetime(vec_cases_new['date']).dt.strftime('%Y/%m/%d')

In [13]:
vec_cases_new = vec_cases_new.rename(columns={'location': 'country'})

In [14]:
vec_cases_new['date'] = vec_cases_new['date'].astype('datetime64[ns]')

## Lockdown data

In [15]:
lockdown = pd.read_excel (r'/content/drive/MyDrive/CS_573_Group_project/Data/OxCGRT_timeseries_all.xlsx')

In [16]:
lockdown_new = lockdown.drop(columns={"country_name","country_code"})
lockdown_new.columns = pd.to_datetime(lockdown_new.columns)

In [17]:
lockdown_input = pd.DataFrame(columns=["country_name","lockdown_index","date"])
for i in range(lockdown_new.shape[1]):
  country = lockdown[{"country_name"}]
  index = lockdown_new.iloc[:,i]
  index_con = pd.concat([country,index], axis=1, ignore_index=True)
  index_con["date"] = index.name
  index_con = index_con.rename(columns={0: 'country_name',  1: 'lockdown_index'})
  lockdown_input = pd.concat([lockdown_input,index_con],ignore_index=True)

In [18]:
lockdown_input = lockdown_input.sort_values('country_name',ascending=True)

In [19]:
lockdown_input = lockdown_input.rename(columns={"country_name":"country"})

## Merge all datasets

In [20]:
df_merged1 = vec_cases_new.merge(mobility_country, on=['country', 'date'])

In [21]:
df_merged1.shape

(97134, 13)

In [22]:
df_merged = df_merged1.merge(lockdown_input, on=['country', 'date'])

In [23]:
df_merged['weekday'] = df_merged['date'].dt.dayofweek

In [26]:
# cases
df_merged = df_merged.sort_values(by=['country', 'date'])
df_merged['new_cases_raw']=df_merged.groupby(['country'])['total_cases'].diff().fillna(0)
df_merged['cases_ppm'] = (df_merged['new_cases_raw'] / df_merged['population'])*1000000

# deaths
df_merged['new_deaths_raw']=df_merged.groupby(['country'])['total_deaths'].diff().fillna(0)
df_merged['deaths_ppm'] = (df_merged['new_deaths_raw'] / df_merged['population'])*1000000

# vaccination cumulative
df_merged['total_vaccinations'] = df_merged.groupby(['country'])['new_people_vaccinated_smoothed'].cumsum()


### test-train-validation split


In [27]:
from sklearn.model_selection import train_test_split
def train_test_split_udf (df, frac):
    X_train, X_test = train_test_split(df, test_size=(frac), random_state=22)
    return X_train, X_test

In [28]:
df_train_1, df_test_1 = train_test_split_udf (df_merged, 0.2)
df_train_2, df_val_1 = train_test_split_udf (df_train_1, 0.125)